In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder as onehot
from sklearn.compose import ColumnTransformer 
import copy
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
#from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix, classification_report
import string
#from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# import torch
# import torch.nn as nn
# from torch.autograd import Variable
# import torchvision.transforms as transforms
# import torchvision.datasets as dsets
from sklearn.decomposition import PCA,TruncatedSVD, IncrementalPCA
from scipy.sparse import random as sparse_random
# from sklearn.random_projection import sparse_random_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
###### METHOD 1 ######

In [11]:


df = pd.read_csv("mod_train.csv")

df['adjusted_price'] = df['price']*df['no of items']
df['adjusted_discount'] = df['adjusted_price'] * df['discount']

df_cp = df.copy()

df_darr = df_cp.drop(['price','discount','no of items','segment','delivery type','year','id','Unnamed: 0','departure city','location','class','RID','departure state','delivery date','placement date','address code'],axis=1)
print(df_darr)



one_hot = pd.get_dummies(df_darr['month'])
# Drop column B as it is now encoded
df_darr = df_darr.drop('month',axis = 1)
# Join the encoded df
df_darr = df_darr.join(one_hot)

one_hot = pd.get_dummies(df_darr['sub-class'])
# Drop column B as it is now encoded
df_darr = df_darr.drop('sub-class',axis = 1)
# Join the encoded df
df_darr = df_darr.join(one_hot, rsuffix = 'sc')

Y = df_darr['profit']
df_darr.drop(['profit'], axis = 1, inplace = True)

train_x, test_x, train_y, test_y = train_test_split(df_darr, Y ,test_size=0.2)


xgb_r = xgb.XGBRegressor(objective ='reg:linear', 
                  n_estimators = 2000, learning_rate = 0.01, max_depth = 15, seed = 123) 
xgb_r.fit(train_x.to_numpy(), train_y.to_numpy()) 
  
# Predict the model 
pred = xgb_r.predict(test_x.to_numpy()) 
  
# RMSE Computation 
#zr = np.zeros()
rmse = 1 - np.sum((pred-test_y)*(pred-test_y))/(np.sum(test_y*test_y)) 
print("RMSE : % f" %(rmse)) 

       sub-class    profit  month  delay  adjusted_price  adjusted_discount
0          phone    5.5512      1      0          32.896             6.5792
1        headset   13.3650      1      0         148.500             0.0000
2         shorts    4.8609      1      0          44.190             0.0000
3     television   12.7368      1      0         131.760             0.0000
4         laptop   15.9792      1      0         133.160             0.0000
...          ...       ...    ...    ...             ...                ...
6695     battery  152.0232      1      0        1628.820             0.0000
6696     headset   89.9548      1      0         553.568           110.7136
6697       watch    0.9588      1      0           2.040             0.0000
6698     tshirts    2.3094      1      0         623.538            62.3538
6699  colddrinks   10.0485      1      0         482.328            96.4656

[6700 rows x 6 columns]
[14:45:25] WARNING: ../src/objective/regression_obj.cu:170: reg

In [13]:
df_darr

,delay,adjusted_price,adjusted_discount,1,2,3,4,5,6,7,...,laptop,lighting,pants,phone,shorts,sweets,tablet,television,tshirts,watch
0,0,32.896,6.5792,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,148.500,0.0000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,44.190,0.0000,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,131.760,0.0000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,133.160,0.0000,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6695,0,1628.820,0.0000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6696,0,553.568,110.7136,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6697,0,2.040,0.0000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6698,0,623.538,62.3538,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
rfr = RandomForestRegressor( n_estimators = 2000, max_depth = 20, random_state= 0) 

rfr.fit(train_x.to_numpy(), train_y.to_numpy()) 
  
# Predict the model 
pred = rfr.predict(test_x.to_numpy()) 
  
# RMSE Computation 
#zr = np.zeros()
rmse = 1 - np.sum((pred-test_y)*(pred-test_y))/(np.sum(test_y*test_y)) 
print("RMSE : % f" %(rmse)) 

RMSE :  0.624747


In [ ]:
###### method 2 #######

In [1]:
def manage_dataset_test(df, upsample, enc1,enc2):
    df_im = df.copy()
    df_im = df_im.drop(['price','no of items','discount','segment','delivery type','year','id','Unnamed: 0','departure city','location','class','RID','departure state','delivery date','placement date','address code'],axis=1)

    train_y_imb = df_im['sub-class']
    train_x_imb = df_im.copy()
    
    print("\n\n\n", df_im.columns)
    
    if upsample == True:
        oversample = SMOTENC(sampling_strategy = 'minority',categorical_features=[0], random_state = 100)

        train_x_b, train_y_b = oversample.fit_resample(train_x_imb,train_y_imb)

        df_darr = train_x_b.copy()
    else:
        df_darr = train_x_imb.copy()
    
    df_arr = df_darr.to_numpy()
    
#     print('df_im : ', df_im)
#     print("df_darr : ", df_darr)
    
    
    
    subcl = np.reshape(np.array(df_arr[:,0],dtype=np.str),(-1,1))

    month = np.reshape(np.array(df_arr[:,2],dtype=np.str),(-1,1))

#     enc1 = onehot(handle_unknown='ignore')


#     enc1.fit(subcl)
    print(enc1.categories_)
    subclv = enc1.transform(subcl).toarray()
    print(subclv)
    
    
#     enc2 = onehot(handle_unknown='ignore')
#     enc2.fit(month)
    print(enc2.categories_)
    monthv = enc2.transform(month).toarray()
    print(monthv)

    up_arr = np.concatenate((monthv[:],subclv[:]),axis=1)

    up_arr = np.concatenate((up_arr,df_arr[:,2:]),axis=1)
    print(up_arr)
    pr_arr = df_arr[:,1]
    print(pr_arr)

    return up_arr,pr_arr, enc1, enc2, subclv,monthv

In [8]:
def manage_dataset_train(df, upsample):
    df_im = df.copy()
    df_im = df_im.drop(['price','no of items','discount','segment','delivery type','year','id','Unnamed: 0','departure city','location','class','RID','departure state','delivery date','placement date','address code'],axis=1)
    
    print("\n\n\n", df_im.columns)
    train_y_imb = df_im['sub-class']
    train_x_imb = df_im.copy()
    
    if upsample == True:
        oversample = SMOTENC(sampling_strategy = 'minority',categorical_features=[0], random_state = 100)

        train_x_b, train_y_b = oversample.fit_resample(train_x_imb,train_y_imb)

        df_darr = train_x_b.copy()
    else:
        df_darr = train_x_imb.copy()
    
    df_arr = df_darr.to_numpy()
    print("\n\n\n", df_darr.columns)
    
#     print('df_im : ', df_im)
#     print("df_darr : ", df_darr)
    
    
    
    subcl = np.reshape(np.array(df_arr[:,0],dtype=np.str),(-1,1))

    month = np.reshape(np.array(df_arr[:,2],dtype=np.str),(-1,1))

    enc1 = onehot(handle_unknown='ignore')


    enc1.fit(subcl)
    print(enc1.categories_)
    subclv = enc1.transform(subcl).toarray()
    print(subclv)
    
    
    
    enc2 = onehot(handle_unknown='ignore')
    enc2.fit(month)
    print(enc2.categories_)
    monthv = enc2.transform(month).toarray()
    print(monthv)

    up_arr = np.concatenate((monthv[:],subclv[:]),axis=1)

    up_arr = np.concatenate((up_arr,df_arr[:,3:]),axis=1)
    print(up_arr)
    pr_arr = df_arr[:,1]
    print(pr_arr)

    return up_arr,pr_arr, enc1, enc2, subclv, monthv

In [9]:
df = pd.read_csv("mod_train.csv")
df['adjusted_price'] = df['price']*df['no of items']
df['adjusted_discount'] = df['adjusted_price'] * df['discount']

In [10]:
from numpy import asarray
from sklearn.preprocessing import StandardScaler


up_arr, pr_arr, enc1, enc2,sc,m = manage_dataset_train(df, False)

print(up_arr.shape, pr_arr.shape)
# up_arr = np.reshape(up_arr,(-1,1))
# scaler2 = StandardScaler()
# pr_arr = scaler2.fit_transform(np.reshape(pr_arr,(-1,1)))
# pr_arr = np.reshape(pr_arr,(-1,1))
train_X, test_X, train_y, test_y = train_test_split(up_arr, pr_arr, 
                      test_size = 0.2, random_state = 123) 
  
# Instantiation 
xgb_r = xgb.XGBRegressor(objective ='reg:linear', 
                  n_estimators = 2000, learning_rate = 0.005, max_depth = 15, seed = 123) 

xgb_r.fit(train_X, train_y) 
  
# Predict the model 

# scaler3 = StandardScaler()


pred = xgb_r.predict(test_X) 
print("Pred_scaled : ", pred)

rmse = 1 - np.sum((pred-test_y)*(pred-test_y))/(np.sum(test_y*test_y)) 
print("RMSE : % f" %(rmse)) 




 Index(['sub-class', 'profit', 'month', 'delay', 'adjusted_price',
       'adjusted_discount'],
      dtype='object')



 Index(['sub-class', 'profit', 'month', 'delay', 'adjusted_price',
       'adjusted_discount'],
      dtype='object')
[array(['battery', 'charger', 'chocolates', 'colddrinks', 'fastfood',
       'headset', 'hoodies', 'laptop', 'lighting', 'pants', 'phone',
       'shorts', 'sweets', 'tablet', 'television', 'tshirts', 'watch'],
      dtype='<U10')]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[array(['1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'],
      dtype='<U2')]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
[[1.0 0.0 0.0 ... 0 32.896 6.5792]
 [1.0 0.0 0.0 ... 0 148.5 0.0]
 [1.0 0.0 0.0 ... 0 44.19 0.0]
 ...
 [1.0 0.0 0.0 ... 0 